In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import svm
from sklearn.model_selection import train_test_split,cross_val_score,KFold
from sklearn.linear_model import Lasso,LinearRegression,LogisticRegression,ElasticNet
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.tree import DecisionTreeClassifier,BaseDecisionTree,DecisionTreeRegressor
from sklearn.ensemble import BaggingClassifier,StackingClassifier,RandomForestClassifier,RandomForestRegressor,GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor

from sklearn.preprocessing import LabelEncoder,StandardScaler,OneHotEncoder

import warnings
# warnings.filterwarnings("ignore")

le = LabelEncoder()
ohe = OneHotEncoder()
sc = StandardScaler()

rfc =  RandomForestClassifier(n_estimators=100)
rfr = RandomForestRegressor(n_estimators=100)
lr = LinearRegression()
lgr = LogisticRegression()
gnb = GaussianNB()
mnb = MultinomialNB()
dtc = DecisionTreeClassifier()
svrc = svm.SVR()
lasso = Lasso()
bagc = BaggingClassifier()

models = [rfc,lr,lgr,gnb,mnb,dtc,svrc,lasso,bagc]

In [2]:
train = pd.read_excel('Data_Train.xlsx')
train.head(10)

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory,Price
0,The Prisoner's Gold (The Hunters 3),Chris Kuzneski,"Paperback,– 10 Mar 2016",4.0 out of 5 stars,8 customer reviews,THE HUNTERS return in their third brilliant no...,Action & Adventure (Books),Action & Adventure,220.00
1,Guru Dutt: A Tragedy in Three Acts,Arun Khopkar,"Paperback,– 7 Nov 2012",3.9 out of 5 stars,14 customer reviews,A layered portrait of a troubled genius for wh...,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts",202.93
2,Leviathan (Penguin Classics),Thomas Hobbes,"Paperback,– 25 Feb 1982",4.8 out of 5 stars,6 customer reviews,"""During the time men live without a common Pow...",International Relations,Humour,299.00
3,A Pocket Full of Rye (Miss Marple),Agatha Christie,"Paperback,– 5 Oct 2017",4.1 out of 5 stars,13 customer reviews,A handful of grain is found in the pocket of a...,Contemporary Fiction (Books),"Crime, Thriller & Mystery",180.00
4,LIFE 70 Years of Extraordinary Photography,Editors of Life,"Hardcover,– 10 Oct 2006",5.0 out of 5 stars,1 customer review,"For seven decades, ""Life"" has been thrilling t...",Photography Textbooks,"Arts, Film & Photography",965.62
5,ChiRunning: A Revolutionary Approach to Effort...,Danny Dreyer,"Paperback,– 5 May 2009",4.5 out of 5 stars,8 customer reviews,The revised edition of the bestselling ChiRunn...,Healthy Living & Wellness (Books),Sports,900.00
6,Death on the Nile (Poirot),Agatha Christie,"Paperback,– 5 Oct 2017",4.4 out of 5 stars,72 customer reviews,Agatha Christie’s most exotic murder mystery\n...,"Crime, Thriller & Mystery (Books)","Crime, Thriller & Mystery",224.00
7,Yoga Your Home Practice Companion: A Complete ...,Sivananda Yoga Vedanta Centre,"Hardcover,– Import, 1 Mar 2018",4.7 out of 5 stars,16 customer reviews,"Achieve a healthy body, mental alertness, and ...",Sports Training & Coaching (Books),Sports,836.00
8,Karmayogi: A Biography of E. Sreedharan,M S Ashokan,"Paperback,– 15 Dec 2015",4.2 out of 5 stars,111 customer reviews,Karmayogi is the dramatic and inspiring story ...,Biographies & Autobiographies (Books),"Biographies, Diaries & True Accounts",130.00
9,"The Iron King (The Accursed Kings, Book 1)",Maurice Druon,"Paperback,– 26 Mar 2013",4.0 out of 5 stars,1 customer review,‘This is the original game of thrones’ George ...,Action & Adventure (Books),Action & Adventure,695.00


In [3]:
print(train['Title'].nunique())
train['Synopsis'].nunique()

5568


5549

In [4]:
train.drop(['Title','Synopsis','Edition'],axis='columns',inplace=True)
train.head()

,Author,Reviews,Ratings,Genre,BookCategory,Price
0,Chris Kuzneski,4.0 out of 5 stars,8 customer reviews,Action & Adventure (Books),Action & Adventure,220.00
1,Arun Khopkar,3.9 out of 5 stars,14 customer reviews,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts",202.93
2,Thomas Hobbes,4.8 out of 5 stars,6 customer reviews,International Relations,Humour,299.00
3,Agatha Christie,4.1 out of 5 stars,13 customer reviews,Contemporary Fiction (Books),"Crime, Thriller & Mystery",180.00
4,Editors of Life,5.0 out of 5 stars,1 customer review,Photography Textbooks,"Arts, Film & Photography",965.62


In [5]:
y = train['Price']
train.isna().sum()

Author          0
Reviews         0
Ratings         0
Genre           0
BookCategory    0
Price           0
dtype: int64

In [6]:
print(train['Reviews'].nunique())
print(train['Ratings'].nunique())
print(train['BookCategory'].nunique())
print(train['Genre'].nunique())
print(train['Author'].nunique())

36
342
11
345
3679


In [7]:
train['Reviews'] = train.Reviews.apply(lambda r: float(r.split()[0]))
train['Ratings']= train.Ratings.str.extract('(\d+)')
train["Ratings"] = train.Ratings.astype(float)
train.head()

,Author,Reviews,Ratings,Genre,BookCategory,Price
0,Chris Kuzneski,4.0,8.0,Action & Adventure (Books),Action & Adventure,220.00
1,Arun Khopkar,3.9,14.0,Cinema & Broadcast (Books),"Biographies, Diaries & True Accounts",202.93
2,Thomas Hobbes,4.8,6.0,International Relations,Humour,299.00
3,Agatha Christie,4.1,13.0,Contemporary Fiction (Books),"Crime, Thriller & Mystery",180.00
4,Editors of Life,5.0,1.0,Photography Textbooks,"Arts, Film & Photography",965.62


In [8]:
dumm1 = pd.get_dummies(train['BookCategory'])
dumm1

,Action & Adventure,"Arts, Film & Photography","Biographies, Diaries & True Accounts",Comics & Mangas,"Computing, Internet & Digital Media","Crime, Thriller & Mystery",Humour,"Language, Linguistics & Writing",Politics,Romance,Sports
0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6232,0,0,0,0,0,0,1,0,0,0,0
6233,0,0,0,0,0,1,0,0,0,0,0
6234,0,0,0,0,0,0,0,0,0,1,0
6235,1,0,0,0,0,0,0,0,0,0,0


In [9]:
xx = pd.concat([train,dumm1],axis='columns')
xx.drop('BookCategory',axis='columns',inplace=True)
xx

,Author,Reviews,Ratings,Genre,Price,Action & Adventure,"Arts, Film & Photography","Biographies, Diaries & True Accounts",Comics & Mangas,"Computing, Internet & Digital Media","Crime, Thriller & Mystery",Humour,"Language, Linguistics & Writing",Politics,Romance,Sports
0,Chris Kuzneski,4.0,8.0,Action & Adventure (Books),220.00,1,0,0,0,0,0,0,0,0,0,0
1,Arun Khopkar,3.9,14.0,Cinema & Broadcast (Books),202.93,0,0,1,0,0,0,0,0,0,0,0
2,Thomas Hobbes,4.8,6.0,International Relations,299.00,0,0,0,0,0,0,1,0,0,0,0
3,Agatha Christie,4.1,13.0,Contemporary Fiction (Books),180.00,0,0,0,0,0,1,0,0,0,0,0
4,Editors of Life,5.0,1.0,Photography Textbooks,965.62,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6232,Tom Phillips,5.0,2.0,Anthropology (Books),322.00,0,0,0,0,0,0,1,0,0,0,0
6233,Stephenie Meyer,3.3,9.0,Contemporary Fiction (Books),421.00,0,0,0,0,0,1,0,0,0,0,0
6234,Julia Quinn,3.8,3.0,Romance (Books),399.00,0,0,0,0,0,0,0,0,0,1,0
6235,Amanda Hocking,3.5,4.0,Action & Adventure (Books),319.00,1,0,0,0,0,0,0,0,0,0,0


In [10]:
xx = xx.astype({'Author':'category','Genre':'category'})
xx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6237 entries, 0 to 6236
Data columns (total 16 columns):
 #   Column                                Non-Null Count  Dtype   
---  ------                                --------------  -----   
 0   Author                                6237 non-null   category
 1   Reviews                               6237 non-null   float64 
 2   Ratings                               6237 non-null   float64 
 3   Genre                                 6237 non-null   category
 4   Price                                 6237 non-null   float64 
 5   Action & Adventure                    6237 non-null   uint8   
 6   Arts, Film & Photography              6237 non-null   uint8   
 7   Biographies, Diaries & True Accounts  6237 non-null   uint8   
 8   Comics & Mangas                       6237 non-null   uint8   
 9   Computing, Internet & Digital Media   6237 non-null   uint8   
 10  Crime, Thriller & Mystery             6237 non-null   uint8   
 11  Humo

In [11]:
xx['Author'] = le.fit_transform(xx['Author'])
xx['Genre'] = le.fit_transform(xx['Genre'])
xx.drop('Author',axis='columns',inplace=True)
xx.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6237 entries, 0 to 6236
Data columns (total 15 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Reviews                               6237 non-null   float64
 1   Ratings                               6237 non-null   float64
 2   Genre                                 6237 non-null   int32  
 3   Price                                 6237 non-null   float64
 4   Action & Adventure                    6237 non-null   uint8  
 5   Arts, Film & Photography              6237 non-null   uint8  
 6   Biographies, Diaries & True Accounts  6237 non-null   uint8  
 7   Comics & Mangas                       6237 non-null   uint8  
 8   Computing, Internet & Digital Media   6237 non-null   uint8  
 9   Crime, Thriller & Mystery             6237 non-null   uint8  
 10  Humour                                6237 non-null   uint8  
 11  Language, Linguis

In [12]:
x = xx.drop('Price',axis='columns')
y

0       220.00
1       202.93
2       299.00
3       180.00
4       965.62
         ...  
6232    322.00
6233    421.00
6234    399.00
6235    319.00
6236    452.00
Name: Price, Length: 6237, dtype: float64

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2)
print(len(x_train))
print(len(y_test))

4989
1248


In [14]:
from sklearn.pipeline import Pipeline
pipelines = []
pipelines.append(('ScaledLR', Pipeline([('Scaler', StandardScaler()),('LR',LinearRegression())])))
pipelines.append(('ScaledLASSO', Pipeline([('Scaler', StandardScaler()),('LASSO', Lasso())])))
pipelines.append(('ScaledEN', Pipeline([('Scaler', StandardScaler()),('EN', ElasticNet())])))
pipelines.append(('ScaledKNN', Pipeline([('Scaler', StandardScaler()),('KNN', KNeighborsRegressor())])))
pipelines.append(('ScaledCART', Pipeline([('Scaler', StandardScaler()),('CART', DecisionTreeRegressor())])))
pipelines.append(('ScaledGBM', Pipeline([('Scaler', StandardScaler()),('GBM', GradientBoostingRegressor())])))

results = []
names = []
for name, model in pipelines:
    kfold = KFold(n_splits=10)
    cv_results = cross_val_score(model, x, y, cv=kfold, scoring='neg_mean_squared_error')
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)


ScaledLR: -429820.172580 (120629.509964)
ScaledLASSO: -429580.698532 (120882.383581)
ScaledEN: -433123.903062 (121531.608234)
ScaledKNN: -481813.291577 (100734.331750)
ScaledCART: -774988.587542 (168031.489317)
ScaledGBM: -415588.857388 (120056.247596)


In [15]:
from sklearn.model_selection import GridSearchCV

scaler = StandardScaler().fit(x)
rescaledX = scaler.transform(x)
param_grid = dict(n_estimators=np.array([50,100,200,300,400]))
model = GradientBoostingRegressor()
kfold = KFold(n_splits=10)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_squared_error', cv=kfold)
grid_result = grid.fit(rescaledX, y)

means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

-417857.187326 (120602.884082) with: {'n_estimators': 50}
-415691.059815 (120158.845980) with: {'n_estimators': 100}
-418942.777926 (123446.642116) with: {'n_estimators': 200}
-423544.564435 (125174.356379) with: {'n_estimators': 300}
-433372.921454 (128403.909834) with: {'n_estimators': 400}
Best: -415691.059815 using {'n_estimators': 100}


In [16]:
from sklearn.metrics import mean_squared_error

scaler = StandardScaler().fit(x)
rescaled_X_train = scaler.transform(x)
model = GradientBoostingRegressor(random_state=21, n_estimators=50)
model.fit(rescaled_X_train, y)


GradientBoostingRegressor(n_estimators=50, random_state=21)

In [17]:
svrc.fit(x_train,y_train)
print(svrc.score(x_test,y_test))
svrc.predict(x[:1])

-0.043551472517352474


array([353.61330156])

In [18]:
rfr.fit(x_train,y_train)
print(rfr.score(x_test,y_test))
rfr.predict(x[:1])

-0.039484629593738774


array([517.19460714])

In [19]:
lr.fit(x_train,y_train)
print(lr.score(x_test,y_test))
print(lr.predict(x[:1]))

from sklearn.pipeline import Pipeline
pipelines = []
pipelines.append(('ScaledLR', Pipeline([('Scaler', StandardScaler()),('LR',LinearRegression())])))

results = []
names = []
for name, model in pipelines:
    kfold = KFold(n_splits=10)
    cv_results = cross_val_score(model, x, y, cv=kfold)
    results.append(cv_results)
    print(results)


0.06988984803617238
[388.19607196]
[array([0.15100067, 0.0778676 , 0.11347918, 0.07865716, 0.05662977,
       0.10415446, 0.12762185, 0.13809305, 0.07715076, 0.08343919])]


In [30]:
gm = GradientBoostingRegressor(n_estimators=100)
gm.fit(x_train,y_train)
print(gm.score(x_test,y_test))
gm.predict(x[:1])

0.0714685365780916


array([360.41119021])

In [21]:
gm.predict(x[:10])

array([ 360.41119021,  331.92866412,  578.58989206,  309.69301314,
       1290.22523213,  717.71832796,  286.35046699,  556.33053745,
        276.00271048,  437.04787027])

In [31]:
from sklearn.metrics import accuracy_score,mean_squared_error,r2_score
y_pred_train = gm.predict(x_train)
print("Train r2_score:",r2_score(y_train, y_pred_train))

Train r2_score: 0.25170868089687604


In [23]:
lasso.fit(x_train,y_train)
print(lasso.score(x_test,y_test))
lasso.predict(x[:1])

0.06998014745018455


array([396.37361705])

In [24]:
test = pd.read_excel('Data_Test.xlsx')
test

,Title,Author,Edition,Reviews,Ratings,Synopsis,Genre,BookCategory
0,The Complete Sherlock Holmes: 2 Boxes sets,Sir Arthur Conan Doyle,"Mass Market Paperback,– 1 Oct 1986",4.4 out of 5 stars,960 customer reviews,A collection of entire body of work of the She...,Short Stories (Books),"Crime, Thriller & Mystery"
1,Learn Docker - Fundamentals of Docker 18.x: Ev...,Gabriel N. Schenker,"Paperback,– Import, 26 Apr 2018",5.0 out of 5 stars,1 customer review,Enhance your software deployment workflow usin...,Operating Systems Textbooks,"Computing, Internet & Digital Media"
2,Big Girl,Danielle Steel,"Paperback,– 17 Mar 2011",5.0 out of 5 stars,4 customer reviews,"'Watch out, world. Here I come!'\nFor Victoria...",Romance (Books),Romance
3,Think Python: How to Think Like a Computer Sci...,Allen B. Downey,"Paperback,– 2016",4.1 out of 5 stars,11 customer reviews,"If you want to learn how to program, working w...",Programming & Software Development (Books),"Computing, Internet & Digital Media"
4,Oxford Word Skills: Advanced - Idioms & Phrasa...,Redman Gairns,"Paperback,– 26 Dec 2011",4.4 out of 5 stars,9 customer reviews,"Learn and practise the verbs, prepositions and...",Linguistics (Books),"Language, Linguistics & Writing"
...,...,...,...,...,...,...,...,...
1555,100 Things Every Designer Needs to Know About ...,Susan Weinschenk,"Paperback,– 14 Apr 2011",5.0 out of 5 stars,4 customer reviews,We design to elicit responses from people. We ...,Design,"Computing, Internet & Digital Media"
1556,"Modern Letter Writing Course: Personal, Busine...",ARUN SAGAR,"Paperback,– 8 May 2013",3.6 out of 5 stars,13 customer reviews,"A 30-day course to write simple, sharp and att...",Children's Reference (Books),"Biographies, Diaries & True Accounts"
1557,The Kite Runner Graphic Novel,Khaled Hosseini,"Paperback,– 6 Sep 2011",4.0 out of 5 stars,5 customer reviews,The perennial bestseller-now available as a se...,Humour (Books),Humour
1558,Panzer Leader (Penguin World War II Collection),Heinz Guderian,"Paperback,– 22 Sep 2009",3.5 out of 5 stars,3 customer reviews,Heinz Guderian - master of the Blitzkrieg and ...,United States History,"Biographies, Diaries & True Accounts"


In [25]:
test.drop(['Title','Synopsis','Edition'],axis='columns',inplace=True)
test['Reviews'] = test.Reviews.apply(lambda r: float(r.split()[0]))
test['Ratings']= test.Ratings.str.extract('(\d+)')
test["Ratings"] = test.Ratings.astype(float)
dumm1 = pd.get_dummies(test['BookCategory'])

yy = pd.concat([test,dumm1],axis='columns')
yy.drop('BookCategory',axis='columns',inplace=True)
yy = yy.astype({'Author':'category','Genre':'category'})

yy['Author'] = le.fit_transform(yy['Author'])
yy['Genre'] = le.fit_transform(yy['Genre'])
yy.drop('Author',axis='columns',inplace=True)
yy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 14 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Reviews                               1560 non-null   float64
 1   Ratings                               1560 non-null   float64
 2   Genre                                 1560 non-null   int32  
 3   Action & Adventure                    1560 non-null   uint8  
 4   Arts, Film & Photography              1560 non-null   uint8  
 5   Biographies, Diaries & True Accounts  1560 non-null   uint8  
 6   Comics & Mangas                       1560 non-null   uint8  
 7   Computing, Internet & Digital Media   1560 non-null   uint8  
 8   Crime, Thriller & Mystery             1560 non-null   uint8  
 9   Humour                                1560 non-null   uint8  
 10  Language, Linguistics & Writing       1560 non-null   uint8  
 11  Politics         

In [32]:
pre = {'Price': gm.predict(yy)}
pre = pd.DataFrame(pre)
pre

,Price
0,420.234373
1,1103.758518
2,556.043494
3,824.411400
4,506.757620
...,...
1555,1183.911377
1556,275.007367
1557,527.835164
1558,410.146990


In [33]:

pre.to_csv('Submmision.csv',index=False)